In [81]:
## importing necessary libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
from random import randrange ##  allows us to randomize numbers library
import time ## time tracker

In [82]:
## This link brings us to Coinbase's support page. 
## Before we scrape all 40 pages, we first need to indentify what we are targeting.
url = "https://status.coinbase.com/history?page=1"

In [83]:
## Pulling down the webpage.
response = requests.get(url)
print(response.status_code)

200


In [84]:
## Making some soup.
soup = BeautifulSoup(response.text, "html.parser")

In [85]:
## Searching for the name of previous incidents, we see they are in a div with data-react-class="HistoryIndex"
## Isolating Incident History div
incident_div = soup.find("div",{'data-react-class': 'HistoryIndex'})

In [86]:
## selecting "data-react-props"
## tightening range to include just incidents
## replacing curly brackets that break the data up by month and are not needed
## splitting the string into values
s = incident_div["data-react-props"]
start = s.find('"months":[') + 10
end = s.find('}]}]', start)
s = s[start:end].replace("{","").replace("}","")
incident_list = list(s.split(',"'))

In [87]:
## all this code does is clean the data up and remove values we don't want as data
## some of the date and time values are stored as strangely formatted variables

clean_incidents = []
for item in incident_list:
     clean_incidents.append(item.replace('"',"")\
                            .replace("\\u003cvar data-var=\'date\'\\u003e","")\
                            .replace("\\u003c/var\\u003e","")\
                            .replace("\\u003cvar data-var=\'time\'\\u003e","")\
                            .replace("\\u003cvar data-var=\'time\'\\u003e","")\
                            .replace("incidents:[","")\
                            .replace("PDT]","PDT")\
                            .replace("PST]","PST"))
## grabbing year
year = "year:"
incident_year = [int(x.replace("year:","")) for x in clean_incidents if year in x]
print(incident_year)
## we will need to add all months in final scrape
##excludes values that contain stats for each month
december = ":December"
january = ":January"
november = ":November"
october = ":October"
september = ":September"
august = ":August"
year = "year:"
starts = "starts_on"
days = "days:"
cleaner_incidents = [ x for x in clean_incidents\
                     if october not in x\
                     and september not in x\
                     and august not in x\
                     and december not in x\
                     and january not in x\
                     and november not in x\
                     and days not in x\
                     and starts not in x\
                     and year not in x]

[2022, 2022, 2022]


In [88]:
name = "name:"
incident_names = [x.replace("name:","") for x in cleaner_incidents if name in x]

In [89]:
message = "message:"
incident_messages = [x.replace("message:","") for x in cleaner_incidents if message in x]

In [90]:
code = "code:"
incident_codes = [x.replace("code:","") for x in cleaner_incidents if code in x]

In [91]:
impact = "impact:"
incident_impacts = [x.replace("impact:","") for x in cleaner_incidents if impact in x]

In [97]:
timestamp = "timestamp:"
incident_timestamps = [x.replace("timestamp:","") for x in cleaner_incidents if timestamp in x]

In [98]:
## creating year column
incident_years = []
jan = "Jan"
for item in incident_timestamps:
    if jan in item:
        incident_years.append(incident_year[0])
    else:
        incident_years.append(incident_year[2])

In [99]:
## creating month column
incident_months = []
jan = "Jan"
feb = "Feb"
mar = "Mar"
apr = "Apr"
may = "May"
jun = "Jun"
jul = "Jul"
aug = "Aug"
sep = "Sep"
oct_ = "Oct"
nov = "Nov"
dec = "Dec"
for item in incident_timestamps:
    if jan in item:
        incident_months.append("January")
    elif feb in item:
        incident_months.append("February")
    elif mar in item:
        incident_months.append("March")
    elif apr in item:
        incident_months.append("April")
    elif may in item:
        incident_months.append("May")
    elif jun in item:
        incident_months.append("June")
    elif jul in item:
        incident_months.append("July")
    elif aug in item:
        incident_months.append("August")
    elif sep in item:
        incident_months.append("September")
    elif oct_ in item:
        incident_months.append("October")
    elif nov in item:
        incident_months.append("November")
    elif dec in item:
        incident_months.append("December")
    else:
        incident_months.append("None")

In [100]:
## compling data into df

incident_dict = []
for (name,impact,timestamp,message,code,year,month) in zip(incident_names,incident_impacts,incident_timestamps,incident_messages,incident_codes,incident_years,incident_months):
    incident_dict.append({
        "Name":name,
        "Impact":impact,
        "Timestamp":timestamp,
        "Message":message,
        "Code":code,
        "Year":year,
        "Month":month
    })
df = pd.DataFrame.from_dict(incident_dict)
df

,Name,Impact,Timestamp,Message,Code,Year,Month
0,Delayed Bitcoin (BTC) Sends and Receives,none,"Oct 9, 20:50 - 22:58 PDT",This incident has been resolved.,hwl4m7f0cs92,2022,October
1,Delayed Filecoin (FIL) Deposits and Withdrawals,none,"Oct 6, 18:35 PDT",We are currently investigating this issue.,0pwb71mk9wk5,2022,October
2,XTZ Withdrawals/Deposits Delayed,none,"Oct 3, 11:55 - 14:25 PDT",This incident has been resolved.,lnpkhb3bx1h6,2022,October
3,iDEAL Status,maintenance,"Aug 18, 08:39 - Oct 3, 12:58 PDT",This incident has been resolved.,sqysq1125v9q,2022,August
4,"ACH withdrawals, deposits, and buys failures",major,"Oct 2, 03:57 - 09:41 PDT",This incident has been resolved.,b45vlknj3cml,2022,October
5,Solana (SOL) Delayed Sends and Receives,minor,"Sep 30, 15:21 - Oct 2, 08:32 PDT",This incident has been resolved.,hs3myh07tvtv,2022,September
6,Delayed Deposits / Withdrawals on Polkadot (DOT),minor,"Sep 28, 04:23 - 07:00 PDT",This incident has been resolved.,7tdy2njx416z,2022,September
7,Solana (Sol) Delayed Sends and Receives,minor,"Sep 26, 00:08 - 02:30 PDT",This incident has been resolved.,7k5yt1mx952n,2022,September
8,Optimism (OP) Delayed Sends and Receives,none,"Sep 24, 23:33 - Sep 25, 19:05 PDT",This incident has been resolved.,tbfm3dmfz2v8,2022,September
9,Polygon Delayed Sends and Recived,none,"Sep 18, 07:34 - Sep 24, 23:30 PDT",This incident has been resolved.,qkn4yc9cmcjq,2022,September


In [128]:
##Now lets make a function that does all of that for us!
def SpecialSauce(response):
    soup = BeautifulSoup(response.text, "html.parser")
    incident_div = soup.find("div",{'data-react-class': 'HistoryIndex'})
    s = incident_div["data-react-props"]
    start = s.find('"months":[') + 10
    end = s.find('}]}]', start)
    s = s[start:end].replace("{","").replace("}","")
    incident_list = list(s.split(',"'))
    clean_incidents = []
    for item in incident_list:
         clean_incidents.append(item.replace('"',"")\
                            .replace("\\u003cvar data-var=\'date\'\\u003e","")\
                            .replace("\\u003c/var\\u003e","")\
                            .replace("\\u003cvar data-var=\'time\'\\u003e","")\
                            .replace("\\u003cvar data-var=\'time\'\\u003e","")\
                            .replace("incidents:[","")\
                            .replace("PDT]","PDT")\
                            .replace("PST]","PST"))
    year = "year:"
    incident_year = [int(x.replace("year:","")) for x in clean_incidents if year in x]
    january = ":January"
    febraury = ":February"
    march = ":March"
    april = ":April"
    may = ":May"
    june = ":June"
    july = ":July"
    august = ":August"
    september = ":September"
    october = ":October"
    november = ":November"
    december = ":December"
    august = ":August"
    year = "year:"
    starts = "starts_on"
    days = "days:"
    cleaner_incidents = [ x for x in clean_incidents\
                     if january not in x\
                     and febraury not in x\
                     and march not in x\
                     and april not in x\
                     and may not in x\
                     and june not in x\
                     and july not in x\
                     and august not in x\
                     and september not in x\
                     and october not in x\
                     and november not in x\
                     and december not in x\
                     and days not in x\
                     and starts not in x\
                     and year not in x]
    name = "name:"
    incident_names = [x.replace("name:","") for x in cleaner_incidents if name in x]
    message = "message:"
    incident_messages = [x.replace("message:","") for x in cleaner_incidents if message in x]
    code = "code:"
    incident_codes = [x.replace("code:","") for x in cleaner_incidents if code in x]
    impact = "impact:"
    incident_impacts = [x.replace("impact:","") for x in cleaner_incidents if impact in x]
    timestamp = "timestamp:"
    incident_timestamps = [x.replace("timestamp:","") for x in cleaner_incidents if timestamp in x]
    incident_years = []
    jan = "Jan"
    for item in incident_timestamps:
        if jan in item:
            incident_years.append(incident_year[0])
        else:
            incident_years.append(incident_year[2])
    incident_months = []
    jan = "Jan"
    feb = "Feb"
    mar = "Mar"
    apr = "Apr"
    may_ = "May"
    jun = "Jun"
    jul = "Jul"
    aug = "Aug"
    sep = "Sep"
    oct_ = "Oct"
    nov = "Nov"
    dec = "Dec"
    for item in incident_timestamps:
        if jan in item:
            incident_months.append("January")
        elif feb in item:
            incident_months.append("February")
        elif mar in item:
            incident_months.append("March")
        elif apr in item:
            incident_months.append("April")
        elif may_ in item:
            incident_months.append("May")
        elif jun in item:
            incident_months.append("June")
        elif jul in item:
            incident_months.append("July")
        elif aug in item:
            incident_months.append("August")
        elif sep in item:
            incident_months.append("September")
        elif oct_ in item:
            incident_months.append("October")
        elif nov in item:
            incident_months.append("November")
        elif dec in item:
            incident_months.append("December")
        else:
            incident_months.append("None")
    incident_dict = []
    for (name,impact,timestamp,message,code,year,month) in zip(incident_names,incident_impacts,incident_timestamps,incident_messages,incident_codes,incident_years,incident_months):
        incident_dict.append({
            "Name":name,
            "Impact":impact,
            "Timestamp":timestamp,
            "Message":message,
            "Code":code,
            "Year":year,
            "Month":month
        })
    df = pd.DataFrame.from_dict(incident_dict)
    return df
    

In [129]:
##creating function to generate links
base_url = "https://status.coinbase.com/history?page="
def generateLinks(base_url,total_pages):
    '''
    Provide the base url with number of pages to generate links
    '''
    links = []
    for number in range(1, total_pages + 1): 
        links.append(f"{base_url}{number}") 
    return links

In [130]:
##creating snoozer
def sleepyTime():
    snoozer = randrange(30,60)
    print(f"snoozing for {snoozer} second before next scrape")
    time.sleep(snoozer)

In [131]:
##creating list processor
def processList(all_dfs, file_name):
    df = pd.concat(all_dfs, ignore_index = True)
    df.to_csv(file_name, encoding = "UTF-8", index = False)
    print(f"{file_name} is in you current folder")
    return df

In [132]:
##creating scraper function
def myScraper(url,total_pages,file_name):
    '''
    Input: List links you would like to scrape, total pages, and file name.
    Output: Final df, list of broken links.
    '''
    my_links = generateLinks(url,total_pages)
    all_dfs = []
    busted_links = []

    counter = 1
    for link in my_links:
        print(f"scraping {counter} of {len(my_links)}")
        counter += 1
        print(f"scraping {link}")
        response = requests.get(link)
        if response.status_code == 200:
            df = SpecialSauce(response)
            all_dfs.append(df)
        else:
            print(f"{link} returned a posted link with response {response.status_code}")
            busted_links.append(link)
        if counter < len(my_links):
            sleepyTime()
        else:
            pass
    final_df = processList(all_dfs,file_name)
    print("all done!")
    return (final_df,busted_links)


In [142]:
url = "https://status.coinbase.com/history?page="
total_pages = 37
file_name = "test.csv"
test_df = myScraper(url,total_pages,file_name)[0]
test_df

scraping 1 of 37
scraping https://status.coinbase.com/history?page=1
snoozing for 41 second before next scrape
scraping 2 of 37
scraping https://status.coinbase.com/history?page=2
snoozing for 56 second before next scrape
scraping 3 of 37
scraping https://status.coinbase.com/history?page=3
snoozing for 37 second before next scrape
scraping 4 of 37
scraping https://status.coinbase.com/history?page=4
snoozing for 59 second before next scrape
scraping 5 of 37
scraping https://status.coinbase.com/history?page=5
snoozing for 33 second before next scrape
scraping 6 of 37
scraping https://status.coinbase.com/history?page=6
snoozing for 36 second before next scrape
scraping 7 of 37
scraping https://status.coinbase.com/history?page=7
snoozing for 54 second before next scrape
scraping 8 of 37
scraping https://status.coinbase.com/history?page=8
snoozing for 48 second before next scrape
scraping 9 of 37
scraping https://status.coinbase.com/history?page=9
snoozing for 34 second before next scrape
s

,Name,Impact,Timestamp,Message,Code,Year,Month
0,Delayed Bitcoin (BTC) Sends and Receives,none,"Oct 9, 20:50 - 22:58 PDT",This incident has been resolved.,hwl4m7f0cs92,2022,October
1,Delayed Filecoin (FIL) Deposits and Withdrawals,none,"Oct 6, 18:35 PDT",We are currently investigating this issue.,0pwb71mk9wk5,2022,October
2,XTZ Withdrawals/Deposits Delayed,none,"Oct 3, 11:55 - 14:25 PDT",This incident has been resolved.,lnpkhb3bx1h6,2022,October
3,iDEAL Status,maintenance,"Aug 18, 08:39 - Oct 3, 12:58 PDT",This incident has been resolved.,sqysq1125v9q,2022,August
4,"ACH withdrawals, deposits, and buys failures",major,"Oct 2, 03:57 - 09:41 PDT",This incident has been resolved.,b45vlknj3cml,2022,October
...,...,...,...,...,...,...,...
906,Pending transfers,none,"Dec 24, 14:37 - Dec 25, 07:20 PST",This incident has been resolved.,xm70l90z4j6r,2013,December
907,Login issues on some browsers.,none,"Dec 20, 16:24 - 16:31 PST",This incident has been resolved.,t2r1xq9lzyt6,2013,December
908,Outage,none,"Dec 10, 20:06 - 20:06 PST",Apologies for the downtime - service has been ...,rp11hz18xb28,2013,December
909,Outage,minor,"Dec 6, 16:40 - 18:09 PST",This incident has been resolved.,wvls2y6scbry,2013,December


In [47]:
test_df = pd.read_csv("test.csv")
test_df[test_df["Impact"] == "critical"]

,Name,Impact,Timestamp,Message,Code,Year,Month
53,Coinbase.com is experiencing connectivity issues,critical,"Aug 3, 13:31 - 14:34 PDT",This incident has been resolved.,h5y9db55bwcq,2022,August
98,Coinbase.com is experiencing connectivity issues,critical,"Jun 21, 00:09 - 01:30 PDT",This incident has been resolved.,mj0ysm01x2nc,2022,June
205,iDEAL and Sofort payments delayed,critical,"Jan 11, 09:34 - 15:36 PST",This incident has been resolved.,rvmx7h9p4jqq,2022,January
235,Sofort and iDEAL Payments Currently Unavailable,critical,"Dec 7, 09:08 - 15:24 PST",This incident has been resolved.,01dwynn34xl0,2021,December
262,Coinbase Card Swipe issues,critical,"Oct 27, 12:38 - 14:31 PDT",This incident has been resolved.,fjtngsrsgx88,2021,October
263,Coinbase.com is experiencing connectivity issues,critical,"Oct 27, 07:18 - 11:13 PDT",This incident has been resolved.,v5dx8y2mtqx2,2021,October
410,"Card purchases, card withdrawals, and PayPal p...",critical,"Jun 3, 12:32 - 13:35 PDT",This incident has been resolved.,9l6j38p5mfs1,2021,June
413,SEPA Payment Service Outage,critical,"Jun 1, 16:35 - 18:38 PDT",This incident has been resolved.,0t5ln2lzhrdk,2021,June
429,Ethereum Berlin Network Upgrade,critical,"Apr 15, 04:50 - 15:27 PDT",This incident has been resolved.,w4fmfqypt630,2021,April
475,We are currently investigating issues with car...,critical,"Feb 22, 04:53 - 05:38 PST",This incident has been resolved.,qnyw0zbqcw0n,2021,February


In [48]:
test_df[test_df["Impact"] == "major"]

,Name,Impact,Timestamp,Message,Code,Year,Month
4,"ACH withdrawals, deposits, and buys failures",major,"Oct 2, 03:57 - 09:41 PDT",This incident has been resolved.,b45vlknj3cml,2022,October
38,International USDC Trading,major,"Jul 19, 09:08 - Aug 23, 07:06 PDT",This incident has been resolved.,j49qg8j0h8hp,2022,July
54,"Coinbase Pay sends for BCH, ETC, AVAX unavailable",major,"Aug 2, 13:26 - 14:37 PDT",This incident has been resolved.,c19pn5pm8h5c,2022,August
88,Coinbase.com is experiencing connectivity issues,major,"Jun 27, 14:27 - 20:47 PDT",This incident has been resolved.,7bh5w0jqf4h3,2022,June
90,Coinbase Pay crypto sends are failing at high ...,major,"Jun 26, 09:58 - Jun 27, 10:20 PDT",This incident has been resolved.,k94rnj3nvkqh,2022,June
...,...,...,...,...,...,...,...
875,We're experiencing a high number of requests,major,"Dec 23, 11:36 - 11:41 PST",This incident has been resolved.,6ppglr77hl8c,2015,December
882,Coinbase Exchange Offline,major,"Sep 2, 16:41 - 16:55 PDT",Exchange is back up.,p8t93bjt6cvv,2015,September
886,Website outage,major,"Jul 13, 04:22 - 05:25 PDT",This incident has been resolved.,jqw9bv00vf2q,2015,July
887,Network Connectivity Issues,major,"Jul 6, 14:22 - 14:54 PDT",We have mitigated issues caused by our upstrea...,mwyzfbr3mvjb,2015,July


In [40]:
test_df[test_df["Name"] == "Coinbase.com is experiencing connectivity issues"]

,Name,Impact,Timestamp,Message,Code,Year,Month
53,Coinbase.com is experiencing connectivity issues,critical,"Aug 3, 13:31 - 14:34 PDT",This incident has been resolved.,h5y9db55bwcq,2022,August
88,Coinbase.com is experiencing connectivity issues,major,"Jun 27, 14:27 - 20:47 PDT",This incident has been resolved.,7bh5w0jqf4h3,2022,June
97,Coinbase.com is experiencing connectivity issues,major,"Jun 23, 11:08 - 11:50 PDT",This incident has been resolved.,wzv8ht040xyn,2022,June
98,Coinbase.com is experiencing connectivity issues,critical,"Jun 21, 00:09 - 01:30 PDT",This incident has been resolved.,mj0ysm01x2nc,2022,June
126,Coinbase.com is experiencing connectivity issues,minor,"Apr 25, 01:13 - 01:51 PDT",This incident has been resolved.,c6h0mn0262hq,2022,April
244,Coinbase.com is experiencing connectivity issues,major,"Nov 23, 16:21 - 17:56 PST",This incident has been resolved.,zsz65cz5clzy,2021,November
261,Coinbase.com is experiencing connectivity issues,major,"Oct 27, 12:40 - 14:54 PDT",This incident has been resolved.,5tv5swyq8jg3,2021,October
263,Coinbase.com is experiencing connectivity issues,critical,"Oct 27, 07:18 - 11:13 PDT",This incident has been resolved.,v5dx8y2mtqx2,2021,October
507,Coinbase.com is experiencing connectivity issues,minor,"Jan 7, 07:50 - 22:13 PST",This incident has been resolved.,3yd0mclly9b4,2021,January
521,Coinbase.com is experiencing connectivity issues,none,"Dec 16, 06:45 - 08:27 PST",This incident has been resolved.,93smh99lxtxz,2020,December


In [41]:
test_df[test_df["Name"] == "Coinbase.com down"]

,Name,Impact,Timestamp,Message,Code,Year,Month
662,Coinbase.com down,critical,"Jun 24, 05:28 - 06:39 PDT",This incident has been resolved.,szjlfcc0zk32,2019,June
